In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [4]:
#Abrimos y revisamos los datos:
df_0 = pd.read_csv('/datasets/geo_data_0.csv')
df_1 = pd.read_csv('/datasets/geo_data_1.csv')
df_2 = pd.read_csv('/datasets/geo_data_2.csv')
print('Base de datos muestra')
print(df_0.head())
print()
print('Informacion variables')
print(df_0.info())
print()
print('tamaño df')
print(df_0.shape)
print()

def print_falt_values(region_num, df_num):   
    print(f"Valores faltantes de {region_num}:\n{df_num.isna().sum()}")
    print("-" * 50)
print_falt_values("Región 0", df_0)
print_falt_values("Región 1", df_1)
print_falt_values("Región 2", df_2)


FileNotFoundError: [Errno 2] No such file or directory: '/datasets/geo_data_0.csv'

Los datos en los Df parecen estar completos, organizados y del tipo correcto, se procederá a separar las variables independientes de las dependientes:

In [3]:
#Procedemos a separar features y target (recordar que id no aporta información para el modelo)
def separe_features_target(df_num):   
    target=df_num['product']
    features = df_num.drop(['product', 'id'], axis = 1)
    return features, target

x_0, y_0 = separe_features_target(df_0)
x_1, y_1 = separe_features_target(df_1)
x_2, y_2 = separe_features_target(df_2)

In [4]:
#Dividimos los conjuntos de entrenamiento y validación
def divide_data(x, y, test_size = .25, valid_size = .5, random_state = 1):
    x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size = test_size, random_state = random_state)
    x_valid, x_test, y_valid, y_test = train_test_split(x_temp, y_temp, test_size = valid_size, random_state = random_state)
    return x_train, x_valid, x_test, y_train, y_valid, y_test

x_0_train, x_0_valid, x_0_test, y_0_train, y_0_valid, y_0_test = divide_data(x_0, y_0)
x_1_train, x_1_valid, x_1_test, y_1_train, y_1_valid, y_1_test = divide_data(x_1, y_1)
x_2_train, x_2_valid, x_2_test, y_2_train, y_2_valid, y_2_test = divide_data(x_2, y_2)

#Confirmamos los tamaños de los 
def print_shape(region, x_train, x_valid, x_test):
    print(f"Tamaño conjunto Entrenamiento {region}: {x_train.shape}")
    print(f"Tamaño conjunto Validación {region}: {x_valid.shape}")
    print(f"Tamaño conjunto Prueba {region}: {x_test.shape}")
    print("-" * 50)

print_shape("Región 0", x_0_train, x_0_valid, x_0_test)
print_shape("Región 1", x_1_train, x_1_valid, x_1_test)
print_shape("Región 2", x_2_train, x_2_valid, x_2_test)



Tamaño conjunto Entrenamiento Región 0: (75000, 3)
Tamaño conjunto Validación Región 0: (12500, 3)
Tamaño conjunto Prueba Región 0: (12500, 3)
--------------------------------------------------
Tamaño conjunto Entrenamiento Región 1: (75000, 3)
Tamaño conjunto Validación Región 1: (12500, 3)
Tamaño conjunto Prueba Región 1: (12500, 3)
--------------------------------------------------
Tamaño conjunto Entrenamiento Región 2: (75000, 3)
Tamaño conjunto Validación Región 2: (12500, 3)
Tamaño conjunto Prueba Región 2: (12500, 3)
--------------------------------------------------


In [5]:
#Entrenamos el modelo para la región 0
model_0 = LinearRegression()
model_0.fit(x_0_train, y_0_train)
pred_0 = model_0.predict(x_0_valid)
rmse_0 = mean_squared_error(y_0_valid, pred_0, squared=False)
#Respuestas región 0
print(f'RMSE de la región 0: {rmse_0}')     
print()
print(f"Media de predicciones de la región 0: {pred_0.mean():.2f}")

RMSE de la región 0: 37.78214551400566

Media de predicciones de la región 0: 92.39


In [6]:
#Definimos la Función
def eval_reg(x_train, y_train, x_valid, y_valid):
    model = LinearRegression()
    model.fit(x_train, y_train)
    pred = model.predict(x_valid)
    rmse = mean_squared_error(y_valid, pred, squared=False)
    mean_pred = pred.mean()
    return model, pred, rmse, mean_pred

#Usamos la función con las 3 regiones 
def print_eval_reg(region, x_train, y_train, x_valid, y_valid):
    model, pred, rmse, mean_pred = eval_reg(x_train, y_train, x_valid, y_valid)
    print(f"{region} RMSE: {rmse:.2f}")
    print(f"{region} Media predicha: {mean_pred:.2f}")
    print("-" * 50)
    return model, pred, rmse, mean_pred

model_0, pred_0, rmse_0, mean_pred_0 = print_eval_reg("Región 0", x_0_train, y_0_train, x_0_valid, y_0_valid)
model_1, pred_1, rmse_1, mean_pred_1 = print_eval_reg("Región 1", x_1_train, y_1_train, x_1_valid, y_1_valid)
model_2, pred_2, rmse_2, mean_pred_2 = print_eval_reg("Región 2", x_2_train, y_2_train, x_2_valid, y_2_valid)


Región 0 RMSE: 37.78
Región 0 Media predicha: 92.39
--------------------------------------------------
Región 1 RMSE: 0.89
Región 1 Media predicha: 68.97
--------------------------------------------------
Región 2 RMSE: 40.02
Región 2 Media predicha: 94.94
--------------------------------------------------



-En la Región 0, en promedio, el modelo se equivoca en 37,780 barriles por pozo y  predice un promedio de 92,390 barriles por pozo.

-En la Región 1, en promedio, el modelo se equivoca en 890 barriles por pozo y  predice un promedio de 68,970 barriles por pozo.

-En la Región 2, en promedio, el modelo se equivoca en 40,020 barriles por pozo y  predice un promedio de 94,940 barriles por pozo.

In [7]:
#Nos preparamos para el calculo de ganancias.
#Almacenamos valores para calculo
inv_tot=100000000
num_wells=200
income_unit=4500
#Calculamos la producción en miles de barriles requerida por pozo
def print_production_avrg(inv_tot, num_wells, income_unit, medias_por_region):
    product_req = inv_tot/num_wells/income_unit
    print(f"La producción media requerida por pozo para cubrir costos debe ser de {product_req:.2f} miles de barriles\n")
    print("-" * 50)
    
    for region, media in medias_por_region.items():
        print(f"Producción promedio {region}: {media:.2f} miles de barriles")
        print("-" * 50)
        
medias_por_region = {
    "Región 0": mean_pred_0,
    "Región 1": mean_pred_1,
    "Región 2": mean_pred_2}

print_production_avrg(inv_tot, num_wells, income_unit, medias_por_region)

    

La producción media requerida por pozo para cubrir costos debe ser de 111.11 miles de barriles

--------------------------------------------------
Producción promedio Región 0: 92.39 miles de barriles
--------------------------------------------------
Producción promedio Región 1: 68.97 miles de barriles
--------------------------------------------------
Producción promedio Región 2: 94.94 miles de barriles
--------------------------------------------------


-Se requiere que en promedio cada pozo produzca 111.11 mil barriles por pozo; sin embargo, las predicciones medias de las tres regiones están debajo de ese objetivo y por lo tanto, ninguna garantiza rentabilidad solo con el promedio.

-Necesitamos seleccionar los 200 mejores pozos con mayor poduccion predicha y ver si logran superar el objetivo de 111.11 mil barriles.

In [8]:
#Calculemos la ganancia esperada de los 200 pozos con mayor produccion estimada
#Comenzaremos haciendo una función que prediga la produccion de barriles de 
#petroleo y el beneficio total esperado del modelo para los 200 pozos más productivos.
def calculate_profit(model, x_test, y_test, sample_size=200, income_per_unit=4500):
    predictions = model.predict(x_test)
    top_prod_well = predictions.argsort()[-sample_size:]
    selected_well = y_test.iloc[top_prod_well]
    total_income = selected_well.sum()*income_per_unit
    return total_income, selected_well, predictions[top_prod_well]

#Resultados por region 
def print_results_region(region, selected_well, total_income, inv_tot):
    print(f"Resultados de {region}:")
    print(f"Producción promedio (200 mejores pozos): {selected_well.mean():.2f} miles de barriles")
    print(f"Ingreso estimado: {total_income:,.2f} USD")
    print(f"Ganancia estimada: {(total_income - inv_tot):,.2f} USD")
    print("-" * 50)
    
#Calculamos los ingresos estimados por el modelo de los 200 mejores pozos por región.
total_income_0, selected_well_0, pred_0_selected = calculate_profit(model_0, x_0_test, y_0_test)
total_income_1, selected_well_1, pred_1_selected = calculate_profit(model_1, x_1_test, y_1_test)
total_income_2, selected_well_2, pred_2_selected = calculate_profit(model_2, x_2_test, y_2_test)

print_results_region("Región 0", selected_well_0, total_income_0, inv_tot)
print_results_region("Región 1", selected_well_1, total_income_1, inv_tot)
print_results_region("Región 2", selected_well_2, total_income_2, inv_tot)



Resultados de Región 0:
Producción promedio (200 mejores pozos): 144.61 miles de barriles
Ingreso estimado: 130,151,186.30 USD
Ganancia estimada: 30,151,186.30 USD
--------------------------------------------------
Resultados de Región 1:
Producción promedio (200 mejores pozos): 137.95 miles de barriles
Ingreso estimado: 124,150,866.97 USD
Ganancia estimada: 24,150,866.97 USD
--------------------------------------------------
Resultados de Región 2:
Producción promedio (200 mejores pozos): 138.52 miles de barriles
Ingreso estimado: 124,667,627.26 USD
Ganancia estimada: 24,667,627.26 USD
--------------------------------------------------


La ganancia que se obtendría si se utilizaran la media de producción de los 200 pozos principales por región incrementaría con respecto al anterior calculo que solo calculaba la produccion media de la región; por lo que bajo ese supuesto, todas las regiones serían rentables al tener una producción estimada mayor de 111.11 miles de barriles, siendo la opción más viable la Región 0, pues la ganancia estimada sería de 30,151,186.30 USD.


Sin embargo este resultado asume que al hacer labores de exploración y explotación petrolera, OilyGiant accederá a los 200 pozos principales, lo cual es poco probable; por lo que se prosigue con el método bootstrapping a buscar la probabilidad de que las ganancias sean positivas.


In [9]:
#Calcularemos riesgos y ganancias para cada región
#Utilizamos la técnica de remuestreo bootstrapping y hallamos las ganancias
def bootstrap_profit(model, x_test, y_test, n_iterations=1000, 
                     sample_size=500, num_wells=200, income_unit=4500, 
                     inv_tot=100000000, random_state=1):

    state = np.random.RandomState(random_state)
    profits = []
#Creamos el bucle para seleccionar 1000 muestras aleatorias de 500 pozos cada una
    for i in range(n_iterations):
        sample_index = state.choice(x_test.index, size=sample_size, replace=True)
        x_subsample = x_test.loc[sample_index]
        y_subsample = y_test.loc[sample_index]
#Predecimos la produccion de los pozos en las muestras aleatorias con el modelo
        predictions = model.predict(x_subsample)
#Elejimos los 200 pozos con mayor produccion esperada
        top_prod_idx = predictions.argsort()[-num_wells:]
        selected_real = y_subsample.iloc[top_prod_idx]
#Calculamos ingreso y lo agreganos a la lista profits[]
        income = selected_real.sum() * income_unit
        profits.append(income)
#Obtenemos los ingresos medios con profits.mean() y el intervalo del 95%
    profits = pd.Series(profits)
    mean_income = profits.mean()
    interval = profits.quantile([0.025, 0.975])
    risk = (profits < inv_tot).mean()

    return mean_income, (interval.loc[0.025], interval.loc[0.975]), risk

In [10]:
mean_0, interval_0, risk_0 = bootstrap_profit(model_0, x_0_test, y_0_test)
mean_1, interval_1, risk_1 = bootstrap_profit(model_1, x_1_test, y_1_test)
mean_2, interval_2, risk_2 = bootstrap_profit(model_2, x_2_test, y_2_test)


In [11]:
def print_bootstrap_results(region_num, mean_income, interval, risk):
    print(f"Resultados bootstrapping de {region_num}")
    print(f"Ingreso promedio: ${mean_income:,.2f}")
    print(f"Intervalo del 95%: ${interval[0]:,.2f} - ${interval[1]:,.2f}")
    print(f"Riesgo de pérdida: {risk:.2%}")
    print("-" * 50)
print_bootstrap_results("Región 0", mean_0, interval_0, risk_0)
print_bootstrap_results("Región 1", mean_1, interval_1, risk_1)
print_bootstrap_results("Región 2", mean_2, interval_2, risk_2)

Resultados bootstrapping de Región 0
Ingreso promedio: $104,210,543.30
Intervalo del 95%: $98,889,725.66 - $109,632,920.17
Riesgo de pérdida: 6.60%
--------------------------------------------------
Resultados bootstrapping de Región 1
Ingreso promedio: $104,848,209.26
Intervalo del 95%: $101,037,160.17 - $108,882,160.77
Riesgo de pérdida: 0.90%
--------------------------------------------------
Resultados bootstrapping de Región 2
Ingreso promedio: $104,155,614.08
Intervalo del 95%: $98,931,008.39 - $109,633,409.31
Riesgo de pérdida: 5.70%
--------------------------------------------------


Posterior a elaborar el método bootstrapping, en donde se hicieron mil iteraciones de 500 muestras para calcular la probabilidad de acceder a los 200 pozos principales, se procedió a calcular el ingreso promedio esperado, el intervalo de confianza del 95% y los riesgos de pérdida.

Bajo esta evaluación, obtenemos que la Región 1 es la más viable para el proyecto de extracción, pues reporta los ingresos promedio esperados más elevados: $104,848,209.26.

Todos los valores del ingreso esperado dentro del intervalo del 95 % de confianza se encuentran por encima del valor de la inversión. Además, el riesgo de pérdida (la probabilidad de que la inversión sea mayor que el ingreso) es el más bajo, con solo un 0.9 % de probabilidad. Esto hace que la Región 1 sea la opción más viable para el proyecto de extracción.

El cambio de decisión sobre la región más viable (de la Región 0 a la Región 1) se debe a que, en la primera evaluación, se trabajó bajo el supuesto de que la empresa OilyGiant lograría identificar y explotar exactamente los 200 pozos más productivos.

Sin embargo, encontrar los pozos más rentables es una cuestión de probabilidad, influida por diversos factores geológicos. Por ello, la Región 1 se vuelve la opción más viable, ya que presenta una mayor probabilidad de contener pozos de alta producción, lo que se traduce en mayores ganancias esperadas y el menor riesgo de pérdida.